In [ ]:
#| default_exp config.mongo

<h1><center>Mongo</center></h1>

In this section, we will define the tools needed to connect and extract data from mongoDB. It will be mainly based on functions from `mongoEngine`

In [ ]:
# | export

import mongoengine
from typing import Optional
from betting_env.config.localconfig import CONFIG, DB_HOSTS


def db_connect(
    db_host: str,  # Host name as defined in `DB_HOSTS`.
    db_name: str,  # Name of the database to connect to.
    db_alias: Optional[
        str
    ] = None,  # alias of the database we are connecting to. If not provided, we will use `db_name`.
):
    "Connect to the apprpriate mongo database."
    # check that the host name provided is valid
    if db_host not in DB_HOSTS:
        raise ValueError(
            "db-host provided {db_host} should be one of {hosts}:".format(
                db_host=db_host, hosts=DB_HOSTS
            )
        )

    # decide on the alias to apply
    db_alias = db_name if not db_name else db_alias

    # read config for the appropriate database
    db_config = CONFIG["databases"][db_host]

    # form the mongo-url i.e check if we need the port
    db_url = (
        db_config["url"]
        if not db_config["port"]
        else db_config["url"] + ":" + db_config["port"]
    )

    db_uri = "{base_url}{user}:{password}@{url}/{db}".format(
        base_url=db_config["mongo_base"],
        user=db_config["user"],
        password=db_config["password"],
        url=db_url,
        db=db_name,
    )
    # add optional argument
    optional_uri = []
    if db_config["majority"]:
        optional_uri.append("w={majority}".format(majority="majority"))
    if db_config["retry_writes"]:
        optional_uri.append(
            "retryWrites={majority}".format(
                majority=str(db_config["retry_writes"]).lower()
            )
        )
    if db_config["authSource"]:
        optional_uri.append(
            "authSource={auth_db}".format(auth_db=db_config["authSource"])
        )

    if optional_uri:
        db_uri += "?" + "&".join(optional_uri)

    mongoengine.register_connection(host=db_uri, alias=db_alias, name=db_name)

Here we register the connection to the database where the `Fixtures` info are saved. It will be used later to extract the game information needed to our RL environment.

In [ ]:
football_con = CONFIG["connections"]["football"]
db_connect(db_host="prod_atlas", db_name=football_con["db"], db_alias="Football")

We will need a function to set-up all the mongoDB connections. It should be called when the package is initialised:

In [ ]:
# | export


def mongo_init(
    db_host: str,  # Host name as defined in `DB_HOSTS`.
):
    "Register all the required mongo connections."
    # check that the host name provided is valid
    if db_host not in DB_HOSTS:
        raise ValueError(
            "db-host provided {db_host} should be one of {hosts}:".format(
                db_host=db_host, hosts=DB_HOSTS
            )
        )

    ## football db
    db_connect(
        db_host=db_host,
        db_name=CONFIG["connections"]["football"]["db"],
        db_alias="football",
    )